In [ ]:
# ==========================================
# 1. 라이브러리 및 드라이브 마운트
# ==========================================
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import drive

# CPU 사용 설정 (GPU 무시)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
drive.mount('/content/drive')

# ==========================================
# 2. 경로 및 하이퍼파라미터 (사용자 확인 필요)
# ==========================================
# 경로에 공백이나 오타가 없는지 다시 한번 확인해 주세요.
BASE_PATH = '/content/drive/MyDrive/빅데이터_부산/이미지1차수정' 
IMG_SIZE = (299, 299)
BATCH_SIZE = 16  # CPU 환경 메모리 고려
NUM_CLASSES = 74

# ==========================================
# 3. 데이터 증강 및 학습/테스트 분할
# ==========================================
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,      # 증강 강도 유지
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2    # 8:2 분할
)

train_generator = datagen.flow_from_directory(
    BASE_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    BASE_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# ==========================================
# 4. ResNet50V2 모델 구축 (전이학습)
# ==========================================
# 사전 학습된 ResNet50V2 가중치 로드
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 가중치 고정

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(512, activation='relu'), # Dense 레이어 크기 상향
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ==========================================
# 5. 모델 학습 시작
# ==========================================
print("\n--- [ResNet50V2] 학습을 시작합니다 ---")
history = model.fit(
    train_generator,
    epochs=25, 
    validation_data=val_generator,
    verbose=1
)

# ==========================================
# 6. Confusion Matrix & 시각화
# ==========================================
print("\n--- 평가 및 Confusion Matrix 생성 중 ---")

# 예측값 생성
Y_pred = model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_generator.classes
class_labels = list(val_generator.class_indices.keys())

# 혼동행렬 생성
cm = confusion_matrix(y_true, y_pred)

# 히트맵 시각화
plt.figure(figsize=(25, 20))
sns.heatmap(cm, annot=False, fmt='d', cmap='YlGnBu', 
            xticklabels=class_labels, yticklabels=class_labels)

plt.title('ResNet50V2 Confusion Matrix', fontsize=20)
plt.ylabel('Actual Label', fontsize=15)
plt.xlabel('Predicted Label', fontsize=15)
plt.xticks(rotation=90)
plt.show()

# 분류 지표 출력
print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=class_labels))
